In [2]:
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

# 난수 시드 설정
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

### 데이터셋과 데이터 로더 작성

In [3]:
train_list = make_datapath_list('train')
val_list = make_datapath_list('val')

size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_dataset = HymenopteraDataset(file_list = train_list, transform = ImageTransform(size, mean, std), phase='train')
val_dataset = HymenopteraDataset(file_list = val_list, transform = ImageTransform(size, mean, std), phase='val')

# DataLoader 작성
batch_size = 32

train_dataloader = data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = data.DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

# 사전 객체에 정리
dataloaders_dict = {'train': train_dataloader, 'val': val_dataloader}

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


### 네트워크 모델 작성

In [4]:
# VGG-16 모델의 인스턴스 생성
use_pretrained = True # 학습된 파라미터 사용
net = models.vgg16(pretrained=use_pretrained)

# VGG의 마지막 출력층의 출력 유닛을 개미와 벌인 두 개로 변경 (전결합 층)
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# 훈련 모드로 설정
net.train()

print('네트워크 설정 완료: 학습된 가중치를 읽어들여 훈련 모드로 설정했습니다.')

네트워크 설정 완료: 학습된 가중치를 읽어들여 훈련 모드로 설정했습니다.


### 손실함수 정의

In [5]:
criterion = nn.CrossEntropyLoss()

### 최적화 기법 설정

In [6]:
# 파인튜닝으로 학습시킬 파라미터를 params_to_update 변수에 저장
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 학습시킬 파라미터 이름
update_param_names_1 = ['features']
update_param_names_2 = ['classifier.0.weight', 'classifier.0.bias', 'classifier.3.weight', 'classifier.3.bias']
update_param_names_3 = ['classifier.6.weight', 'classifier.6.bias']

# 파라미터를 각 리스트에 저장
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print('params_to_update_1에 저장: ', name)
    
    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print('params_to_update_2에 저장: ', name)
    
    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print('params_to_update_3에 저장: ', name)
        
    else:
        param.requires_grad = False
        print('경사 계산 없음. 학습하지 않음: ', name)
        

params_to_update_1에 저장:  features.0.weight
params_to_update_1에 저장:  features.0.bias
params_to_update_1에 저장:  features.2.weight
params_to_update_1에 저장:  features.2.bias
params_to_update_1에 저장:  features.5.weight
params_to_update_1에 저장:  features.5.bias
params_to_update_1에 저장:  features.7.weight
params_to_update_1에 저장:  features.7.bias
params_to_update_1에 저장:  features.10.weight
params_to_update_1에 저장:  features.10.bias
params_to_update_1에 저장:  features.12.weight
params_to_update_1에 저장:  features.12.bias
params_to_update_1에 저장:  features.14.weight
params_to_update_1에 저장:  features.14.bias
params_to_update_1에 저장:  features.17.weight
params_to_update_1에 저장:  features.17.bias
params_to_update_1에 저장:  features.19.weight
params_to_update_1에 저장:  features.19.bias
params_to_update_1에 저장:  features.21.weight
params_to_update_1에 저장:  features.21.bias
params_to_update_1에 저장:  features.24.weight
params_to_update_1에 저장:  features.24.bias
params_to_update_1에 저장:  features.26.weight
params_to_update_1

In [7]:
# 최적화 기법 설정
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)

### 학습 및 검증 실시

In [8]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    
    # 초기 설정
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print('사용 장치: ', device)
    
    # 네트워크를 GPU로
    net.to(device)
    
    # 네트워크가 어느정도 고정되면 고속화시킨다.
    torch.backends.cudnn.benchmark = True
    
    # 에폭 루프
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('--------------------------------')
        
        # 에폭별 학습 및 검증 루프
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()
                
            epoch_loss = 0.0 # 에폭 손실 합
            epoch_corrects = 0 # 에폭 정답 수
            
            # 학습하지 않을 시 검증 성능을 확인하기 위해 epoch=0의 훈련은 생략
            if (epoch==0) and (phase=='train'):
                continue
                
            # 데이터로더로 미니 배치를 꺼내는 루프
            for inputs, labels in tqdm(dataloaders_dict[phase]):
                # GPU에 데이터를 보낸다.
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                # 순전파 계산
                with torch.set_grad_enabled(phase=='train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    
                    # 훈련시에는 오차 역전파
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                    # 반복 결과 계산
                    # 손실 합계 갱신
                    epoch_loss += loss.item()*inputs.size(0)
                    # 정답 수의 합계 갱신
                    epoch_corrects += torch.sum(preds==labels.data)
                    
            # 에폭당 손실과 정답률 표시
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

In [10]:
# 학습 및 검증 실시
num_epochs = 2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs)

사용 장치:  cuda:0
Epoch 1/2
--------------------------------


100%|██████████| 5/5 [00:01<00:00,  3.47it/s]


val Loss: 0.1260 Acc: 0.9542
Epoch 2/2
--------------------------------


100%|██████████| 8/8 [00:03<00:00,  2.32it/s]


train Loss: 0.0824 Acc: 0.9630


100%|██████████| 5/5 [00:01<00:00,  3.71it/s]

val Loss: 0.1131 Acc: 0.9542


### 학습한 네트워크 저장 및 로드